In [4]:
mgpt_tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT")
mgpt_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

In [49]:
import torch
import jieba

def get_utt_logp_jieba(utterance, tokenizer, model):
    '''
    Compute log probabilities for each Jieba token instead of fixed-length characters.
    '''
    result = []

    jieba_tokens = list(jieba.cut(utterance))   # ['那个', '故事', '很', '有趣']


    encoded_tokens = tokenizer.encode(utterance, add_special_tokens=False) ## model.tokenizer

    print(encoded_tokens) # encoded_tokens = [6929, 702, 3125, 752, 2523, 3300, 6637]

    with torch.no_grad():
        lm_output = model(torch.tensor(encoded_tokens).unsqueeze(0))  # lm_output.logit contains probabilities for each token.

    lm_logp = torch.log_softmax(lm_output.logits, dim=-1) # Converts model logits into log probabilities using log_softmax()

    char_ls, char_logp_ls = [], []
    token_index = 0  # Track position in encoded_tokens[]

    for jieba_token in jieba_tokens: # '那个'
        char_ls, char_logp_ls  = [], []

        token_len = len(jieba_token)  # Dynamically set chunk size ### token_len == 2

        for _ in range(token_len):
            if token_index >= len(encoded_tokens):
                break
            t = encoded_tokens[token_index] ## t = 6929
            char_ls.append(t) ## char_ls [6929]

            if token_index == 0:
                char_logp_ls.append(None) ## char_logp_ls[None]
                #char_logp_ls.append(lm_logp[0, 0, t].item())
            else:
                char_logp_ls.append(lm_logp[0, token_index - 1, t].item()) ## Skipped for first token and for next : char_logp_ls = [None, lm_logp[0, 0, 702]]

            token_index += 1

        # Decode the grouped characters back into text
        char = "".join([tokenizer.decode([t]).strip() for t in char_ls]) #
        char_logp = None if None in char_logp_ls else sum(char_logp_ls)

        surprisal = None if char_logp is None else -char_logp

        result.append((char, char_logp, surprisal))

    return result


In [56]:
sentences = ["那个故事很有趣"]
result = get_utt_logp_jieba(sentences[0], mgpt_tokenizer, mgpt_model)

for token, logp, surprisal in result:
    print(f"Token: {token}, Log Probability: {logp}, Surprisal : {surprisal}")

[15002, 4998, 35789, 16213, 1924, 40848]
Token: 那个, Log Probability: None, Surprisal : None
Token: 故事很, Log Probability: -11.029080390930176, Surprisal : 11.029080390930176
Token: 有, Log Probability: -1.5473893880844116, Surprisal : 1.5473893880844116
Token: 趣, Log Probability: -1.5169832706451416, Surprisal : 1.5169832706451416


In [62]:
# sentences = ["他做事情总是马虎"]
# result = get_utt_logp_jieba(sentences[0], mgpt_tokenizer, mgpt_model)

# for token, logp, surprisal in result:
#     print(f"Token: {token}, Log Probability: {logp}, Surprisal : {surprisal}")

In [51]:
gpt2_chinese_tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")
gpt2_chinese_model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

In [52]:
sentences = ["那个故事很有趣"]
result = get_utt_logp_jieba(sentences[0], gpt2_chinese_tokenizer, gpt2_chinese_model)

for token, logp, surprisal in result:
    print(f"Token: {token}, Log Probability: {logp}, Surprisal : {surprisal}")

[6929, 702, 3125, 752, 2523, 3300, 6637]
Token: 那个, Log Probability: None, Surprisal : None
Token: 故事, Log Probability: -6.651256904006004, Surprisal : 6.651256904006004
Token: 很, Log Probability: -4.286985397338867, Surprisal : 4.286985397338867
Token: 有趣, Log Probability: -3.3635138273239136, Surprisal : 3.3635138273239136


In [63]:
# sentences = ["他做事情总是马虎"]
# result = get_utt_logp_jieba(sentences[0], gpt2_chinese_tokenizer, gpt2_chinese_model)

# for token, logp, surprisal in result:
#     print(f"Token: {token}, Log Probability: {logp}, Surprisal : {surprisal}")

In [64]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel

# Load tokenizer and model
gpt2_chinese_tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")
gpt2_chinese_model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

# Define the sentence
sentence = "那个故事很有趣"
#sentence = "他做事情总是马虎"

# Tokenize and get input IDs
input_ids = gpt2_chinese_tokenizer.encode(sentence, return_tensors="pt")
tokens = gpt2_chinese_tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())

# Get model predictions
with torch.no_grad():
    outputs = gpt2_chinese_model(input_ids)
    logits = outputs.logits  # Logits from the model output

# Compute token-wise surprisals
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

# Calculate surprisal for each token
surprisals = []
token_ids = input_ids.squeeze().tolist()

for i in range(1, len(token_ids)):  # Start from index 1 (the first token has no context)
    token = tokens[i]  # Correctly mapped token from input IDs
    prob = log_probs[0, i - 1, token_ids[i]].item()  # Get log probability
    surprisal = -prob  # Negative log probability
    surprisals.append((token, surprisal))

# Print results
for token, surprisal in surprisals:
    print(f"Token: {token}, Surprisal: {surprisal:.4f}")


Token: 那, Surprisal: 6.1929
Token: 个, Surprisal: 2.1816
Token: 故, Surprisal: 7.3291
Token: 事, Surprisal: 0.0245
Token: 很, Surprisal: 2.2402
Token: 有, Surprisal: 2.2148
Token: 趣, Surprisal: 1.6255
Token: [SEP], Surprisal: 2.4742


In [1]:
import torch
import jieba
from transformers import AutoTokenizer, AutoModelForCausalLM

mgpt_tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT")
mgpt_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")


# Define the sentence
sentence = "मैं अच्छी तरह से हिंदी नहीं बोलता"  # I don't speak hindi well
#sentence = "他做事情总是马虎"

# Tokenize and get input IDs
input_ids = mgpt_tokenizer.encode(sentence, return_tensors="pt")
tokens = mgpt_tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())

# Get model predictions
with torch.no_grad():
    outputs = mgpt_model(input_ids)
    logits = outputs.logits  # Logits from the model output

# Compute token-wise surprisals
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

# Calculate surprisal for each token
surprisals = []
token_ids = input_ids.squeeze().tolist()

for i in range(1, len(token_ids)):  # Start from index 1 (the first token has no context)
    token = tokens[i]  # Correctly mapped token from input IDs
    prob = log_probs[0, i - 1, token_ids[i]].item()  # Get log probability
    surprisal = -prob  # Negative log probability
    surprisals.append((token, surprisal))

# Print results
for token, surprisal in surprisals:
    print(f"Token: {token}, Surprisal: {surprisal:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Token: à¥Īà¤Ĥ, Surprisal: 6.5619
Token: Ġà¤ħ, Surprisal: 5.0657
Token: à¤ļ, Surprisal: 1.3521
Token: à¥į, Surprisal: 0.0903
Token: à¤Ľ, Surprisal: 0.0002
Token: à¥Ģ, Surprisal: 0.9701
Token: Ġà¤¤à¤°à¤¹, Surprisal: 0.4248
Token: Ġà¤¸, Surprisal: 0.3117
Token: à¥ĩ, Surprisal: 0.0131
Token: Ġà¤¹, Surprisal: 4.1295
Token: à¤¿à¤Ĥ, Surprisal: 5.0702
Token: à¤¦, Surprisal: 0.1867
Token: à¥Ģ, Surprisal: 0.0863
Token: Ġà¤¨à¤¹, Surprisal: 3.8053
Token: à¥Ģà¤Ĥ, Surprisal: 0.0123
Token: Ġà¤¬, Surprisal: 0.9769
Token: à¥ĭ, Surprisal: 0.0031
Token: à¤²à¤¤, Surprisal: 0.7160
Token: à¤¾, Surprisal: 0.6038


In [1]:
## Let's assume we have a Chinese sentence and we want to compute surprisal values for each word

import jieba

sentence = "我想去看那个人"
########################
# 我：I
# 想：want
# 去：go
# 看：see
# 那个：that
# 人：person
########################

# Tokenize the sentence
tokens = jieba.lcut(sentence)
print(tokens)
# Output: ['我', '想', '去', '看', '那个', '人']

# Compute surprisal using pre-tokenized sequence

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer and model
model_name = "Qwen/Qwen2-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()
words = ['我', '想', '去', '看', '那个', '人']

for i in range(len(words)):  # Iterate over each word as target
    context = ''.join(words[:i])  # Exclude target word
    target_word = words[i]  # Current target word

    if context:  # If there is context, tokenize it
        context_ids = tokenizer.encode(context, return_tensors="pt")
    else:  # If no context, use an empty input or BOS token if required
        context_ids = torch.tensor([[tokenizer.bos_token_id]]) if tokenizer.bos_token_id else torch.tensor([[0]])

    # Get logits from model (forward pass)
    with torch.no_grad():
        outputs = model(input_ids=context_ids)
    # Extract last-step probabilities
    logits = outputs.logits[:, -1, :]
    probabilities = torch.softmax(logits, dim=-1)

    # Get token ID of target word
    target_id = tokenizer.encode(target_word, add_special_tokens=False)[0]

    # Compute surprisal (negative log-probability)
    surprisal = -torch.log(probabilities[0, target_id])

    print(f"Word: {target_word}, Context: '{context}', Surprisal: {surprisal.item()}")

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 2.062 seconds.
DEBUG:jieba:Loading model cost 2.062 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['我', '想', '去', '看', '那个', '人']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Word: 我, Context: '', Surprisal: 7.4561004638671875
Word: 想, Context: '我', Surprisal: 8.462255477905273
Word: 去, Context: '我想', Surprisal: 5.128419399261475
Word: 看, Context: '我想去', Surprisal: 10.605973243713379
Word: 那个, Context: '我想去看', Surprisal: 6.8878912925720215
Word: 人, Context: '我想去看那个', Surprisal: 9.31408405303955
